## **NLP Practical**

### **LSTM**

It is now time to train a Recurrent Neural Nework.

#### **Libraries**

We import the necessary libraries for the notebook.

In [1]:
# general
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

# dataset
from torch.utils.data import Dataset, DataLoader

# pytorch
import torch
import torch.nn.functional as F
import torch.nn as nn

# metrics
from sklearn.metrics import mean_squared_error

print("> Libraries Imported")

> Libraries Imported


#### **Import the dataset**

We read our pickle with the encoded texts. 

In [2]:
dataframe = pd.read_pickle("../data/3_multi_eurlex_encoded.pkl")
dataframe.head()

,celex_id,labels,labels_new,text_en,text_de,text_it,text_pl,text_sv,text_en_enc,text_de_enc,text_it_enc,text_pl_enc,text_sv_enc,set
0,32006D0213,1,1,commission decision of _number_ march _number_...,entscheidung der kommission vom _number_ marz ...,decisione della commissione del _number_ marzo...,decyzja komisji z dnia _number_ marca _number_...,kommissionens beslut av den _number_ mars _num...,"[[2, 3, 4, 5, 6, 5, 7, 8, 9, 4, 10, 11, 12, 13...","[[2, 3, 4, 5, 6, 7, 6, 8, 9, 3, 10, 11, 12, 13...","[[2, 3, 4, 5, 6, 7, 6, 8, 9, 10, 11, 12, 13, 1...","[[2, 3, 4, 5, 6, 7, 6, 8, 9, 10, 11, 12, 13, 1...","[[2, 3, 4, 5, 6, 7, 6, 8, 9, 10, 11, 12, 13, 1...",train
1,32003R1786,3,3,council regulation ec no _number_ _number_ of ...,verordnung eg nr _number_ _number_ des rates v...,regolamento ce n _number_ _number_ del consigl...,rozporzadzenie rady we nr _number_ _number_ z ...,radets forordning eg nr _number_ _number_ av d...,"[[133, 177, 34, 92, 5, 5, 4, 5, 178, 5, 44, 8,...","[[196, 31, 98, 6, 6, 42, 43, 5, 6, 197, 6, 49,...","[[219, 41, 110, 6, 6, 5, 53, 5, 6, 220, 6, 221...","[[242, 49, 39, 32, 6, 6, 4, 5, 6, 243, 6, 8, 1...","[[49, 189, 38, 33, 6, 6, 4, 5, 6, 190, 6, 8, 5...",train
2,32004R1038,3,3,commission regulation ec no _number_ _number_ ...,verordnung eg nr _number_ _number_ der kommiss...,regolamento ce n _number_ _number_ della commi...,rozporzadzenie komisji we nr _number_ _number_...,kommissionens forordning eg nr _number_ _numbe...,"[[2, 177, 34, 92, 5, 5, 4, 5, 74, 5, 665, 8, 2...","[[196, 31, 98, 6, 6, 3, 4, 5, 6, 454, 6, 8, 78...","[[219, 41, 110, 6, 6, 3, 4, 5, 6, 471, 6, 8, 8...","[[242, 3, 39, 32, 6, 6, 4, 5, 6, 552, 6, 8, 10...","[[2, 189, 38, 33, 6, 6, 4, 5, 6, 442, 6, 8, 72...",train
3,32003R1012,2,2,commission regulation ec no _number_ _number_ ...,verordnung eg nr _number_ _number_ der kommiss...,regolamento ce n _number_ _number_ della commi...,rozporzadzenie komisji we nr _number_ _number_...,kommissionens forordning eg nr _number_ _numbe...,"[[2, 177, 34, 92, 5, 5, 4, 5, 352, 5, 688, 14,...","[[196, 31, 98, 6, 6, 3, 4, 5, 6, 402, 6, 8, 80...","[[219, 41, 110, 6, 6, 3, 4, 5, 6, 417, 6, 418,...","[[242, 3, 39, 32, 6, 6, 4, 5, 6, 497, 6, 8, 11...","[[2, 189, 38, 33, 6, 6, 4, 5, 6, 397, 6, 8, 77...",train
4,32003R2229,18,11,council regulation ec no _number_ _number_ of ...,verordnung eg nr _number_ _number_ des rates v...,regolamento ce n _number_ _number_ del consigl...,rozporzadzenie rady we nr _number_ _number_ z ...,radets forordning eg nr _number_ _number_ av d...,"[[133, 177, 34, 92, 5, 5, 4, 5, 43, 5, 690, 94...","[[196, 31, 98, 6, 6, 42, 43, 5, 6, 44, 6, 8, 8...","[[219, 41, 110, 6, 6, 5, 53, 5, 6, 54, 6, 8, 4...","[[242, 49, 39, 32, 6, 6, 4, 5, 6, 51, 6, 8, 11...","[[49, 189, 38, 33, 6, 6, 4, 5, 6, 52, 6, 8, 78...",train


#### **Get the splits**

We already divided the dataset in 3 splits, so we can just use the information in the *set* column.

In [3]:
# train 
dataframe_train = dataframe.loc[dataframe["set"] == "train"]
X_train = list(dataframe_train["text_en_enc"])
Y_train = list(dataframe_train["labels_new"])

# validation 
dataframe_val = dataframe.loc[dataframe["set"] == "validation"]
X_val = list(dataframe_val["text_en_enc"])
Y_val = list(dataframe_val["labels_new"])

# test 
dataframe_test = dataframe.loc[dataframe["set"] == "test"]
X_test = list(dataframe_test["text_en_enc"])
Y_test = list(dataframe_test["labels_new"])

In [4]:
# X_temp = list(dataframe['text_en_enc'])
# Y_temp = list(dataframe['labels_new'])

# from sklearn.model_selection import train_test_split
# X_train_temp, X_valid_temp, y_train_temp, y_valid_temp = train_test_split(X_temp, Y_temp, test_size=0.2)

**SOLUZIONE TEMPORANEA**

In [5]:
# from sklearn.model_selection import train_test_split

# X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=1)

# X_train

#### **Create a Pytorch dataset**

We create a custom class that inherits from the pytorch Dataset class. 

In [6]:
class TextsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [7]:
train_dataset = TextsDataset(X_train, Y_train)
valid_dataset = TextsDataset(X_val, Y_val)

# DEBUG
# train_dataset_temp = TextsDataset(X_train_temp, y_train_temp)
# valid_dataset_temp = TextsDataset(X_valid_temp, y_valid_temp)
# DEBUG

**Hyperparameters Setup**

In [8]:
BATCH_SIZE = 10
EPOCHS = 30

VOCAB_SIZE = 78549      # number of words in EN_WORDS
LEARNING_RATE = 0.01

EMBEDDING_DIM=500
HIDDEN_DIM=50
NUM_CLASSES=12

Finally, we can use dataloaders.

In [9]:
train_dl = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dl = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

In [10]:
# train_dl = DataLoader(train_dataset_temp, batch_size=BATCH_SIZE, shuffle=True)
# valid_dl = DataLoader(valid_dataset_temp, batch_size=BATCH_SIZE)

#### **Create training function**

In [11]:
def train_model(model, epochs=10, lr=0.001):

    # setup loss and optimizers
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)

    # for each epoch...
    for i in range(epochs):

        print(f"> EPOCH {i+1} started")

        # training mode
        model.train()

        # reset loss and total
        sum_loss = 0.0
        total = 0

        for x, y, l in tqdm(train_dl):

            # input preprocess
            x = x.long()
            y = y.long()

            # forward pass
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)

            # backwards pass
            loss.backward()
            optimizer.step()

            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]

        # evaluate the model on the validation set
        val_loss, val_acc, val_rmse = validation_metrics(model, valid_dl)

        print(f"> EPOCH {i+1} completed")
        print(f" - Training Loss        {sum_loss/total}")
        print(f" - Validation Loss      {val_loss}")
        print(f" - Validation Accuracy  {val_acc}")
        print(f" - Validation RMSE      {val_rmse}")
        print("=========================================")


def validation_metrics (model, valid_dl):

    # evaluation mode (no need for backwards propagation)
    model.eval()

    # setup placeholders
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0

    for x, y, l in valid_dl:

        # input preprocess
        x = x.long()
        y = y.long()

        # obtain predictions
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]

        # update results
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
        
    return sum_loss/total, correct/total, sum_rmse/total

#### **LSTM with fixed length input**

In [12]:
class LSTM_fixed_len(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):

        # calls the constructor of the parent (nn.Module) 
        # --> so that any initialization done in the super class is still done
        super().__init__()

        # embedding layer
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Linear layer
        self.linear = nn.Linear(hidden_dim, num_classes)
        
        # Dropout Layer
        #self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, l):

        # Set initial hidden and cell states 
        x = self.embeddings(x)
        #x = self.dropout(x)

        # Forward propagate LSTM
        lstm_out, (ht, ct) = self.lstm(x)

        # Decode the hidden state of the last time step and return
        return self.linear(ht[-1])

Now we can instantiate the model.

In [13]:
LSTM_fixed = LSTM_fixed_len(
    vocab_size=VOCAB_SIZE, 
    embedding_dim=EMBEDDING_DIM, 
    hidden_dim=HIDDEN_DIM,
    num_classes=NUM_CLASSES
    )

#### **Train the model**

In [14]:
train_model(LSTM_fixed, epochs=EPOCHS, lr=LEARNING_RATE)

> EPOCH 1 started


  5%|▍         | 90/1973 [01:14<25:49,  1.22it/s]


KeyboardInterrupt: 